In [113]:
import pandas as pd
import re
import datetime

pd.set_option('display.max_colwidth', None)
data = pd.read_csv('jeopardy.csv')
data = data.rename(columns = {' Air Date': 'Air Date', ' Round': 'Round', ' Category': 'Category', ' Value':'Value', ' Question': 'Question', ' Answer':'Answer'})
data['float_val'] = data.Value.apply(lambda x: None if x.replace("$", "").replace(",", "") == 'None' else float(x.replace("$", "").replace(",", "")))
# data['Date_tmp'] = datetime.datetime.strptime(data['Air Date'], '%Y-%m-%d')
data['Date_tmp'] = data['Air Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
print(type(data['Air Date'][1]))
print(type(data['Date_tmp'][1]))
data.describe()
print(data.head())


<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
   Show Number    Air Date      Round                         Category Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY  $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES  $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...  $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE  $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES  $200   

                                                                                                      Question  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state has a record average of 4,055 hours of sunshine each year  

In [102]:
def word_in_question(data, word_list):
#     word_filter = lambda x: all(word.lower() in x.lower() for word in word_list)
#     word_filter = lambda x: all(re.search("\\b"+word.lower()+"('|\\b)", x.lower()) for word in word_list)
    word_filter = lambda x: all(re.search("\\b"+word.lower()+"\\b", x.lower()) for word in word_list)
    return data.loc[data.Question.apply(word_filter)]

test_result = word_in_question(data, ['King', 'England'])
test_result_king = word_in_question(data, ['King'])
print(test_result_king.float_val.mean())
print(test_result_king.describe())

817.6001483679526
       Show Number    float_val
count  2744.000000  2696.000000
mean   4152.577259   817.600148
std    1412.658238   636.201440
min       3.000000   100.000000
25%    3220.750000   400.000000
50%    4313.500000   600.000000
75%    5314.250000  1000.000000
max    6296.000000  9000.000000


In [103]:
test_result_king.groupby('Answer').Value.count().nlargest(5)

Answer
Henry VIII     54
Louis XIV      31
Solomon        31
David          29
Richard III    24
Name: Value, dtype: int64

In [112]:
computer_question = word_in_question(data, ['computer'])
year = computer_question['Date_tmp'].apply(lambda x: x.year)
decade = computer_question['Date_tmp'].apply(lambda x: "80s - 90s" if x.year < 2000 else '00s')
computer_question.groupby(decade).Answer.count().reset_index()


,Date_tmp,Answer
0,00s,278
1,80s - 90s,84


In [114]:
data.groupby(['Round', 'Category']).Question.count().reset_index()

,Round,Category,Question
0,Double Jeopardy!,"""-ARES""",5
1,Double Jeopardy!,"""...OD"" WORDS",5
2,Double Jeopardy!,"""1"", ""2"", ""3""",5
3,Double Jeopardy!,"""20"" QUESTIONS",5
4,Double Jeopardy!,"""A"" + 4",5
...,...,...,...
31681,Jeopardy!,“NORTH”,5
31682,Jeopardy!,“STREETS”,5
31683,Tiebreaker,CHILD'S PLAY,1
31684,Tiebreaker,LITERARY CHARACTERS,1


In [119]:
data.Category.nunique()

27995